# Extraindo padrões dos dados com clustering 🧩

A utilização da técnica de clustering ou agrupamento se relaciona à área da aprendizagem de máquina não supervisionada, na qual não dispõe-se de rótulos que permitem realizar a classificação de uma classe ou previsão de um valor.

Justamente a partir da ausência de um rótulo, compreende-se a necessidade de outras abordagens para a compreensão dos dados, debruçando-se sobre, geralmente, a similaridade entre eles.

Compreendendo que dados similares costumam estar próximos uns dos outros, a integração dessa proximidade forma grupos, por meio dos quais possibilita o agrupamento dos dados.

Não obstante, a compreensão da similaridade dos dados entorno de si ocorre de duas principais formas: a distância e densidade. Nesse sentido, diversos algoritmos surgem, como forma de agrupar os dados com base nessas características, sendo os principais o K-Means, o Mean-Shift e o DBScan, por exemplo.

>

💢 Os detalhes de cada qual já foram explorados noutro notebook desse repositório, então não irei comentar sobre.

>

Com base nisso, o presente estudo desse notebook se refere a compreensão de padrões nos dados utilizando a técnica de clustering, permitindo identificar os rótulos formados e a interpretá-los. O estudo de caso a ele associado é um conjunto de dados que apresenta o comportamento dos usuários de crédito de uma fictícia empresa de cartão de crédito, com o objetivo de agrupá-los entorno das características que apresentam, por meio de sua similaridade.

## Importando as bibliotecas 📚

In [1]:
import pandas as pd
import numpy as np

from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import silhouette_score


In [2]:
df = pd.read_csv('/content/Customer_Data.csv')

df.head()

,cust_id,balance,balance_frequency,purchases,oneoff_purchases,installments_purchases,cash_advance,purchases_frequency,oneoff_purchases_frequency,purchases_installments_frequency,cash_advance_frequency,cash_advance_trx,purchases_trx,credit_limit,payments,minimum_payments,prc_full_payment,tenure
0,C10001,40.900749,0.818182,95.40,0.00,95.4,0.000000,0.166667,0.000000,0.083333,0.000000,0,2,1000.0,201.802084,139.509787,0.000000,12
1,C10002,3202.467416,0.909091,0.00,0.00,0.0,6442.945483,0.000000,0.000000,0.000000,0.250000,4,0,7000.0,4103.032597,1072.340217,0.222222,12
2,C10003,2495.148862,1.000000,773.17,773.17,0.0,0.000000,1.000000,1.000000,0.000000,0.000000,0,12,7500.0,622.066742,627.284787,0.000000,12
3,C10004,1666.670542,0.636364,1499.00,1499.00,0.0,205.788017,0.083333,0.083333,0.000000,0.083333,1,1,7500.0,0.000000,NaN,0.000000,12
4,C10005,817.714335,1.000000,16.00,16.00,0.0,0.000000,0.083333,0.083333,0.000000,0.000000,0,1,1200.0,678.334763,244.791237,0.000000,12


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8950 entries, 0 to 8949
Data columns (total 18 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   cust_id                           8950 non-null   object 
 1   balance                           8950 non-null   float64
 2   balance_frequency                 8950 non-null   float64
 3   purchases                         8950 non-null   float64
 4   oneoff_purchases                  8950 non-null   float64
 5   installments_purchases            8950 non-null   float64
 6   cash_advance                      8950 non-null   float64
 7   purchases_frequency               8950 non-null   float64
 8   oneoff_purchases_frequency        8950 non-null   float64
 9   purchases_installments_frequency  8950 non-null   float64
 10  cash_advance_frequency            8950 non-null   float64
 11  cash_advance_trx                  8950 non-null   int64  
 12  purcha

In [4]:
df.shape

(8950, 18)

In [5]:
def verificaDataFrame(df):

  qt_n_disponivel = df.isna().sum().any()
  qt_nulos = df.isnull().sum().any()
  qt_duplicados = df.duplicated().sum().any()

  if qt_n_disponivel:

    print(f'Há dados não disponíveis no DataFrame.')

  if qt_nulos:

    print(f'Há dados nulos no DataFrame.')

  if qt_duplicados:

    print(f'Há dados duplicados no DataFrame.')

  if not any([qt_n_disponivel, qt_nulos, qt_duplicados]):

    print("""Não há dados duplicados, nulos ou não disponíveis em seu DataFrame.
Sinta-se à vontade para utilizá-los em seu projeto e/ou estudo.""")

In [6]:
verificaDataFrame(df)

Há dados não disponíveis no DataFrame.
Há dados nulos no DataFrame.


In [7]:
df.isna().sum()

cust_id                               0
balance                               0
balance_frequency                     0
purchases                             0
oneoff_purchases                      0
installments_purchases                0
cash_advance                          0
purchases_frequency                   0
oneoff_purchases_frequency            0
purchases_installments_frequency      0
cash_advance_frequency                0
cash_advance_trx                      0
purchases_trx                         0
credit_limit                          1
payments                              0
minimum_payments                    313
prc_full_payment                      0
tenure                                0
dtype: int64

In [8]:
df.isnull().sum()

cust_id                               0
balance                               0
balance_frequency                     0
purchases                             0
oneoff_purchases                      0
installments_purchases                0
cash_advance                          0
purchases_frequency                   0
oneoff_purchases_frequency            0
purchases_installments_frequency      0
cash_advance_frequency                0
cash_advance_trx                      0
purchases_trx                         0
credit_limit                          1
payments                              0
minimum_payments                    313
prc_full_payment                      0
tenure                                0
dtype: int64

Dado a proporcionalidade dos dados em relação aos dados nulos (que aqui são equivalentes aos dados NaN) eu poderia simplesmente excluí-los, mas realizei um pré-tratamento diferente. Ao invés da exclusão, irei substituir os valores nulos pela mediana dos dados a coluna que pertencem.   

In [9]:
df = df.fillna(df.median)

In [10]:
df.isna().sum()

cust_id                             0
balance                             0
balance_frequency                   0
purchases                           0
oneoff_purchases                    0
installments_purchases              0
cash_advance                        0
purchases_frequency                 0
oneoff_purchases_frequency          0
purchases_installments_frequency    0
cash_advance_frequency              0
cash_advance_trx                    0
purchases_trx                       0
credit_limit                        0
payments                            0
minimum_payments                    0
prc_full_payment                    0
tenure                              0
dtype: int64

In [11]:
df.head()

,cust_id,balance,balance_frequency,purchases,oneoff_purchases,installments_purchases,cash_advance,purchases_frequency,oneoff_purchases_frequency,purchases_installments_frequency,cash_advance_frequency,cash_advance_trx,purchases_trx,credit_limit,payments,minimum_payments,prc_full_payment,tenure
0,C10001,40.900749,0.818182,95.40,0.00,95.4,0.000000,0.166667,0.000000,0.083333,0.000000,0,2,1000.0,201.802084,139.509787,0.000000,12
1,C10002,3202.467416,0.909091,0.00,0.00,0.0,6442.945483,0.000000,0.000000,0.000000,0.250000,4,0,7000.0,4103.032597,1072.340217,0.222222,12
2,C10003,2495.148862,1.000000,773.17,773.17,0.0,0.000000,1.000000,1.000000,0.000000,0.000000,0,12,7500.0,622.066742,627.284787,0.000000,12
3,C10004,1666.670542,0.636364,1499.00,1499.00,0.0,205.788017,0.083333,0.083333,0.000000,0.083333,1,1,7500.0,0.000000,<bound method NDFrame._add_numeric_operations....,0.000000,12
4,C10005,817.714335,1.000000,16.00,16.00,0.0,0.000000,0.083333,0.083333,0.000000,0.000000,0,1,1200.0,678.334763,244.791237,0.000000,12


In [12]:
# Removendo colunas que não são pertinentes:

df = df.drop(['cust_id', 'tenure'], axis = 1)

df.head()

,balance,balance_frequency,purchases,oneoff_purchases,installments_purchases,cash_advance,purchases_frequency,oneoff_purchases_frequency,purchases_installments_frequency,cash_advance_frequency,cash_advance_trx,purchases_trx,credit_limit,payments,minimum_payments,prc_full_payment
0,40.900749,0.818182,95.40,0.00,95.4,0.000000,0.166667,0.000000,0.083333,0.000000,0,2,1000.0,201.802084,139.509787,0.000000
1,3202.467416,0.909091,0.00,0.00,0.0,6442.945483,0.000000,0.000000,0.000000,0.250000,4,0,7000.0,4103.032597,1072.340217,0.222222
2,2495.148862,1.000000,773.17,773.17,0.0,0.000000,1.000000,1.000000,0.000000,0.000000,0,12,7500.0,622.066742,627.284787,0.000000
3,1666.670542,0.636364,1499.00,1499.00,0.0,205.788017,0.083333,0.083333,0.000000,0.083333,1,1,7500.0,0.000000,<bound method NDFrame._add_numeric_operations....,0.000000
4,817.714335,1.000000,16.00,16.00,0.0,0.000000,0.083333,0.083333,0.000000,0.000000,0,1,1200.0,678.334763,244.791237,0.000000


In [13]:
df.head()

,balance,balance_frequency,purchases,oneoff_purchases,installments_purchases,cash_advance,purchases_frequency,oneoff_purchases_frequency,purchases_installments_frequency,cash_advance_frequency,cash_advance_trx,purchases_trx,credit_limit,payments,minimum_payments,prc_full_payment
0,40.900749,0.818182,95.40,0.00,95.4,0.000000,0.166667,0.000000,0.083333,0.000000,0,2,1000.0,201.802084,139.509787,0.000000
1,3202.467416,0.909091,0.00,0.00,0.0,6442.945483,0.000000,0.000000,0.000000,0.250000,4,0,7000.0,4103.032597,1072.340217,0.222222
2,2495.148862,1.000000,773.17,773.17,0.0,0.000000,1.000000,1.000000,0.000000,0.000000,0,12,7500.0,622.066742,627.284787,0.000000
3,1666.670542,0.636364,1499.00,1499.00,0.0,205.788017,0.083333,0.083333,0.000000,0.083333,1,1,7500.0,0.000000,<bound method NDFrame._add_numeric_operations....,0.000000
4,817.714335,1.000000,16.00,16.00,0.0,0.000000,0.083333,0.083333,0.000000,0.000000,0,1,1200.0,678.334763,244.791237,0.000000


In [14]:
# Filtrando as linhas que não contêm métodos
df = df[df.applymap(lambda x: not callable(x)).all(axis=1)]

Analisando como os dados estão dispostos no dataframe, compreende-se que, para passar para um modelo de A. não Supervisionado, devemos normalizá-los ou padronizá-los, tendo em vista que é necessário que estejam sujeitos a uma mesma escala. Devido a robustez a outliers, irei normalizar os dados, ao invés de padronizar. A diferença entre um e outro já expliquei no notebook Introdução ao K-Means e ao PCA.

In [15]:
# Instanciando o normalizador
# e o aplicando no conjunto de dados
scaler = StandardScaler()

X = df.values

valores_normalizados = scaler.fit_transform(X)

## Criando o modelo do KMeans

In [16]:
# Criando o modelo :

# n_clusters : quantidade de grupos esperados.

# n_init : forçamos o modelo ser executado 10 vezes
#          para que ele atinja o mesmo resultado, buscando
#          com isso fomentar a sua confiabilidade.

# max_iter : número máximo de interações que o algoritmo
#            irá fazer.

kmeans = KMeans(n_clusters = 8, n_init = 10, max_iter = 300)

y_pred = kmeans.fit_predict(valores_normalizados)

## Avaliando o modelo com a métrica silhueta

$s(i) = \frac{b(i) - a(i)}{\max\{a(i), b(i)\}}$

>

A métrica da silhueta é um método de avaliação que mensura o quão bom um modelo de agrupamento está em compactar os dados, deixá-los coeso dentro de um grupo, e separá-los, de modo a aferir se os grupos agrupados estão bem separados e determinados.

Apresenta um intervalo de valor máximo e mínimo que vai do 1 ao -1, sendo 1 para perfeitamente agrupado, -1 para não agrupado e 0 para casos que o modelo não consegue discernir se os dados estão ou não separados. Desse modo, quanto mais os valores fiquem próximos de 1 melhor

Para o seu cálculo, como a fórmula permite saber, calcula-se a distância média entre os pontos e todos os demais outros de outro grupo (b) - relativo à separação - e a distância média entre todos os pontos dentro do mesmo grupo (a) - relativo à coesão -. Em seu denominador, com o valor das distâncias, analisa qual é máximo, se for o valor de a, o mantém e, se for de b, o mantém.

In [17]:
labels = kmeans.labels_

silhueta = metrics.silhouette_score(valores_normalizados,
                                    labels,
                                    metric = 'euclidean')

print(f'A silhueta do modelo é de aproximadamente {silhueta.round(3)}')

A silhueta do modelo é de aproximadamente 0.241


## Avaliando com o I. de Davis Bouldin

$DB = \frac{1}{n} \sum_{i=1}^{n} \max_{j \neq i} \left(\frac{\sigma_i + \sigma_j}{d(c_i, c_j)}\right)$

Índice que busca avaliar a qualidade de um cluster, considerando nível de sua coesão e separação. A sua principal diferença em relação à silhueta é que ele considera os centróides para a realização do seu cálculo, fato que o permite ser mais resistente a presença de outliers.

Em geral, valores do índice que sejam baixos indicam que os cluster gerados estão coesos e bem separados, ao passo que o oposto indica o inverso.

In [18]:
dbs = metrics.davies_bouldin_score(valores_normalizados,
                                   labels)

print(f'índice Davis Bouldin : {dbs.round(3)}')

índice Davis Bouldin : 1.305


## I. de Calinski Harabas

$CH = \frac{B/(k-1)}{W/(n-k)}$

Trata-se de um coeficiente que mensura a razão de dispersão dos dados entre clusters e intra clusters. Em geral, valores altos tendem a dizer que os grupos agrupados pelo modelo apresentam coesão e estão bem separados. Também utiliza os centróides para o seu cálculo.

In [19]:
calinski = metrics.calinski_harabasz_score(valores_normalizados, labels)

print(f'Valor do índice de Calinski : {calinski.round(3)}')

Valor do índice de Calinski : 1435.612


Calculei os diferentes valores para aferir o modelo, a sua eficiência em separar os grupos a partir dos dados existentes no dataset, porém, em termos práticos, como saber o quão bom ele se encontra com base nas métricas ?

## Validando o modelo

Essa etapa visa responder a pergunta anterior, que se consiste em utilizar das métricas para validar se o modelo é ou não efetivo. Para isso, uma vez que com os valores únicos não nos prove grande informação, irei compará-los com alguns modelos criados, no qual terá o hiperparâmetro da quantidade de clusters variando entre si.

Nesse sentido, aquele modelo que apresentar as melhores métricas para cada métrica, será o selecionado.

In [20]:
def clustering(n_clusters, dataset):

  kmeans = KMeans(n_clusters = n_clusters,
                  n_init = 10, max_iter = 300)

  labels = kmeans.fit_predict(dataset)
  silhueta = metrics.silhouette_score(dataset, labels, metric = 'euclidean')
  dbs = metrics.davies_bouldin_score(dataset, labels)
  calinski = metrics.calinski_harabasz_score(dataset, labels)

  print(f'Métricas \n\n Silhueta : {silhueta.round(3)} \n Davis Bouldin : {dbs.round(3)} \n Calinski : {calinski.round(3)}')

In [21]:
clustering(3, valores_normalizados)

Métricas 

 Silhueta : 0.264 
 Davis Bouldin : 1.53 
 Calinski : 1673.034


In [22]:
clustering(5, valores_normalizados)

Métricas 

 Silhueta : 0.211 
 Davis Bouldin : 1.475 
 Calinski : 1576.996


In [23]:
clustering(8, valores_normalizados)

Métricas 

 Silhueta : 0.241 
 Davis Bouldin : 1.31 
 Calinski : 1435.1


In [24]:
clustering(10, valores_normalizados)

Métricas 

 Silhueta : 0.227 
 Davis Bouldin : 1.327 
 Calinski : 1345.921


In [25]:
clustering(12, valores_normalizados)

Métricas 

 Silhueta : 0.228 
 Davis Bouldin : 1.397 
 Calinski : 1265.217


Analisando as diferentes métricas geradas por meio dos diversos aplicados, nota-se que o melhor modelo é aquele que apresenta o número de 8 clusters, uma vez que ele apresenta os melhores valores comparativos para cada métrica, com a exceção da silhueta que tem o seu melhor valor com a quantidade de 3 clusters, mas que representa, por outro lado, uma diferença pequena em relação ao modelo escolhido.

## Comparando com dados aleatórios :     

Uma outra forma de validar o modelo criado é passar a ele um conjunto de dados aleatório com a mesma dimensionalidade do dataset a ele informado, de modo que se ele apresentar os mesmos ou similares valores acerca de suas métricas, significa que o modelo está encontrando padrões aleatórios, e não uma estrutura real, isto é, ele pouco vale ou é impreciso.

In [26]:
# Gerando o conjunto de dados aleatório,
# com base na dimensionalidade do dataframe
# utilizado no modelo.
random_data = np.random.rand(8950,16)

# Chamando a função que agrupa os dados
# e os mensura nas métricas informadas.
print('Para os dados aleatórios : \n')
clustering(8, random_data)
print('')
print('Para os dados do estudo de caso : \n')
clustering(8, valores_normalizados)

Para os dados aleatórios : 

Métricas 

 Silhueta : 0.04 
 Davis Bouldin : 3.077 
 Calinski : 246.825

Para os dados do estudo de caso : 

Métricas 

 Silhueta : 0.228 
 Davis Bouldin : 1.449 
 Calinski : 1394.985


Como podemos ver, o nosso modelo apresenta valores dissonantes em relação aos dados aleatórios e aos dados do case. Nota-se que o valor de Davis Bouldin é menor (que é o que essa métrica busca, tendo em vista que quanto melhor for, melhor compreende-se que está o modelo) e tanto o índice de Calinski quanto da silhueta é significativamente maior.

Portanto, utilizando-se dessa outra técnica de validação, compreende-se que o modelo está lidando com dados que possuem padrões e é capaz de extrair os grupos desse.

## Estabilidade dos clusters

Outra abordagem para aferir a eficiência do modelo, considerando as pertinentes métricas, é verificar a estabilidade do modelo. Em termos práticos, significa dividir o conjunto de dados em 'n' partes e passar cada uma ao modelo e, posteriormente, as métricas de avaliação.

No que tange ao presente notebook, significa passar os diferentes datasets gerados por meio da segmentação do dataset principal e a quantidade de cluster desejada para a função que instancia o modelo, treina e prevê, sendo posteriormente validado.

In [27]:
set1, set2, set3, set4, set5 = np.array_split(valores_normalizados, 5)

print('Para o set 1 : \n')
clustering(8, set1)

print('')

print('Para o set 2 : \n')
clustering(8, set2)

print('')

print('Para o set 3 : \n')
clustering(8, set3)

print('')

print('Para o set 4 : \n')
clustering(8, set4)

print('')

print('Para o set 5 : \n')
clustering(8, set5)

Para o set 1 : 

Métricas 

 Silhueta : 0.208 
 Davis Bouldin : 1.427 
 Calinski : 350.295

Para o set 2 : 

Métricas 

 Silhueta : 0.22 
 Davis Bouldin : 1.412 
 Calinski : 280.423

Para o set 3 : 

Métricas 

 Silhueta : 0.234 
 Davis Bouldin : 1.322 
 Calinski : 292.23

Para o set 4 : 

Métricas 

 Silhueta : 0.246 
 Davis Bouldin : 1.292 
 Calinski : 321.163

Para o set 5 : 

Métricas 

 Silhueta : 0.251 
 Davis Bouldin : 1.339 
 Calinski : 343.257


Analisando as métricas para cada segmento do datasaframe, nota-se que as métricas, cada qual, em cada porção, apresenta valores similares, o que significa que o modelo utilizado é valido.

## Interpretando os clusters

Geramos os clusters para o dataframe utilizado, relativo ao estudo de caso, porém como conseguimos visualizar e interpretar da melhor forma os grupos formados ? Com a quantidade de dimensões presentes no dataframe original, esse processo se torna difícil, uma vez que os clusters ficarão sobrepostos e difíceis de observar visualmente num gráfico por exemplo.

Dessa forma, há uma abordagem que pode ser seguida para extrairmos as colunas, que é selecionar as colunas que apresentam a maior variância, ou seja, a maior variação dos dados.

Mas por que escolher aqueles que apresentam a maior variância ? A variância se relaciona a quanto varia os dados, certo ? Desse modo, uma menor variância indica que os dados apresentam pouca variação entorno de si, ao passo que uma maior variância indica o oposto. Nesse sentido, as colunas que apresentam maior variância são mais significativas para destacar as características que mais diferenciam o cluster, tornando a sua análise mais elucidativa.


Para compreendermos o valor da variância de cada coluna, precisamos extrair o valor dos centróides e o relacionar com os valores das colunas do dataframe.

In [28]:
centroids = kmeans.cluster_centers_

centroids

array([[-1.32461867e-02,  3.53544453e-01, -3.73139628e-01,
        -2.56211482e-01, -4.11284288e-01, -6.91538244e-02,
        -9.00710230e-01, -4.27033082e-01, -7.83861986e-01,
         1.61913442e-01, -1.23635300e-02, -5.01327177e-01,
        -3.43839832e-01, -2.76381926e-01, -5.07343644e-02,
        -4.68189681e-01],
       [-6.96739423e-01, -2.22865521e+00, -3.07540201e-01,
        -2.31460457e-01, -3.01638569e-01, -3.06523705e-01,
        -5.57812857e-01, -4.29452446e-01, -4.62543504e-01,
        -4.92034136e-01, -3.61795628e-01, -4.25548648e-01,
        -2.06295844e-01, -1.78570647e-01, -2.97322179e-01,
         3.85670247e-01],
       [ 8.28890640e-01,  4.46504610e-01,  2.23207544e+00,
         1.74844608e+00,  2.06414228e+00, -2.04139006e-01,
         1.14748026e+00,  1.59756515e+00,  1.20403153e+00,
        -3.23356794e-01, -2.21333548e-01,  2.71584382e+00,
         1.21844036e+00,  1.27862172e+00,  2.75955471e-01,
         3.12356721e-01],
       [-1.66492072e-01,  3.53861281e

In [29]:
max = len(centroids[0])
for i in range(max):
    print(df.columns.values[i],"\n{:.4f}\n".format(centroids[:, i].var()))

balance 
0.8497

balance_frequency 
0.7368

purchases 
13.4192

oneoff_purchases 
11.9881

installments_purchases 
5.4135

cash_advance 
0.5410

purchases_frequency 
0.6174

oneoff_purchases_frequency 
1.1147

purchases_installments_frequency 
0.5422

cash_advance_frequency 
0.5670

cash_advance_trx 
0.4902

purchases_trx 
3.6022

credit_limit 
1.1526

payments 
7.0353

minimum_payments 
14.0725

prc_full_payment 
0.2715



Observando o valor das variâncias para cada coluna, encontramos as respectivas colunas selecionadas. Ainda que há outras com maior valor de variância, elas não foram selecionadas, pois representam uma variação das colunas selecionadas.

>

### Clusters selecionados:

- purchases
- cash_advance
- credit_limit
- payments
- balance


### Relembrando...

- balance = limite disponível do cliente ;
- purchases = valor total de compras ;
- cash_advance = valor total de saques ;
- credit_limit = limite total de crédito ;
- payments = valor total pago .


## Analisando as colunas selecionadas

In [30]:
df['cluster'] = labels

In [31]:
df.head()

,balance,balance_frequency,purchases,oneoff_purchases,installments_purchases,cash_advance,purchases_frequency,oneoff_purchases_frequency,purchases_installments_frequency,cash_advance_frequency,cash_advance_trx,purchases_trx,credit_limit,payments,minimum_payments,prc_full_payment,cluster
0,40.900749,0.818182,95.40,0.00,95.40,0.000000,0.166667,0.000000,0.083333,0.00,0,2,1000.0,201.802084,139.509787,0.000000,0
1,3202.467416,0.909091,0.00,0.00,0.00,6442.945483,0.000000,0.000000,0.000000,0.25,4,0,7000.0,4103.032597,1072.340217,0.222222,6
2,2495.148862,1.000000,773.17,773.17,0.00,0.000000,1.000000,1.000000,0.000000,0.00,0,12,7500.0,622.066742,627.284787,0.000000,3
4,817.714335,1.000000,16.00,16.00,0.00,0.000000,0.083333,0.083333,0.000000,0.00,0,1,1200.0,678.334763,244.791237,0.000000,0
5,1809.828751,1.000000,1333.28,0.00,1333.28,0.000000,0.666667,0.000000,0.583333,0.00,0,8,1800.0,1400.057770,2407.246035,0.000000,4


In [39]:
# Criando uma descrição agrupada por meio do
# cluster que permite analisar o valor médio
# das colunas do dataframe pelo número total
# de clientes.

description = df.groupby('cluster')[['balance', 'purchases', 'cash_advance',
                                    'credit_limit', 'payments', 'minimum_payments']]

n_clients = description.size()
description = description.mean().round(2)
description['n_clients'] = n_clients

description

,balance,purchases,cash_advance,credit_limit,payments,minimum_payments,n_clients
cluster,,,,,,,
0,1572.65,216.94,847.02,3263.794727,980.15,743.659739,2832
1,141.24,359.00,343.94,3767.248654,1264.90,158.929188,1156
2,3338.12,5862.30,561.13,8980.399061,5504.81,1518.989699,426
3,1252.35,1924.59,296.32,5825.069378,1960.58,485.591663,1140
4,762.01,881.92,231.95,3338.188327,1037.80,618.692358,2126
5,5567.14,24957.91,1858.84,15570.0,25178.88,3475.059479,30
6,4967.85,537.24,5200.50,8161.016343,4024.78,1737.407744,890
7,4250.15,918.27,976.90,4476.388889,1357.09,27995.061876,36


In [40]:
df.groupby('cluster')['prc_full_payment'].describe()

,count,mean,std,min,25%,50%,75%,max
cluster,,,,,,,,
0,2832.0,0.020777,0.060306,0.0,0.000000,0.000000,0.000000,0.666667
1,1156.0,0.273560,0.362017,0.0,0.000000,0.090909,0.500000,1.000000
2,426.0,0.251840,0.378488,0.0,0.000000,0.000000,0.448051,1.000000
3,1140.0,0.262738,0.360451,0.0,0.000000,0.000000,0.500000,1.000000
4,2126.0,0.256123,0.342796,0.0,0.000000,0.083333,0.500000,1.000000
5,30.0,0.478409,0.417721,0.0,0.083333,0.375000,0.916667,1.000000
6,890.0,0.039320,0.109498,0.0,0.000000,0.000000,0.000000,0.916667
7,36.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000


## Analisando os clusters

- Cluster 0:

Tamanho: Grande (2832 clientes)

Inadimplência: Baixíssima (média de 0.02, com 75% dos clientes abaixo de 0.0).

Conclusão: Clientes com baixo risco de inadimplência.

- Cluster 1:

Tamanho: Médio (1156 clientes)

Inadimplência: Média (variando de 0 a 1, com 50% dos clientes tendo um valor de 0.09).

Conclusão: Clientes com risco moderado de inadimplência.

- Cluster 2:

Tamanho: Pequeno (426 clientes)

Inadimplência: Média (variando de 0 a 1, com 50% dos clientes abaixo de 0).

Conclusão: Grupo heterogêneo em relação à inadimplência.

- Cluster 3:

Tamanho: Médio (1140 clientes)

Inadimplência: Média, similar ao Cluster 2.

Conclusão: Grupo heterogêneo em relação à inadimplência.

- Cluster 4:

Tamanho: Grande (2126 clientes)

Inadimplência: Média, tendendo para baixo.

Conclusão: Clientes com risco de inadimplência um pouco abaixo da média.

- Cluster 5:

Tamanho: Muito pequeno (30 clientes)

Inadimplência: Alta (média de 0.47, com 25% dos clientes acima de 0.91).

Conclusão: Possível grupo de alto risco, mas precisa de mais investigação devido ao tamanho reduzido.

- Cluster 6:

Tamanho: Médio (890 clientes)

Inadimplência: Muito baixa, similar ao Cluster 0.

Conclusão: Clientes com baixo risco de inadimplência.

- Cluster 7:

Tamanho: Muito pequeno (36 clientes)

Inadimplência: Nula (todos os valores são 0)

Conclusão: Clientes com baixíssimo risco, provavelmente nunca atrasaram pagamentos. Precisa de mais investigação devido ao tamanho reduzido.
